# Executing commands and Playbooks at Nodes and using the Role System

This notebook briefly describes how to execute commands in CLAP nodes. 
There are three ways to execute commands:
* Directly executing shell commands
* Executing an Ansible playbook
* Executing role's actions

In [15]:
import sys
sys.path.append('../..')
!pip list

Package               Version
--------------------- -----------
ansible               5.8.0
ansible-core          2.12.6
ansible-runner        2.2.0
anyio                 3.6.1
argcomplete           2.0.0
argon2-cffi           21.3.0
argon2-cffi-bindings  21.2.0
asttokens             2.0.5
attrs                 21.4.0
Babel                 2.10.1
backcall              0.2.0
bcrypt                3.2.2
beautifulsoup4        4.11.1
bleach                5.0.0
boto                  2.49.0
boto3                 1.23.9
botocore              1.26.9
certifi               2022.5.18.1
cffi                  1.15.0
charset-normalizer    2.0.12
click                 8.1.3
coloredlogs           15.0.1
contextlib2           21.6.0
cryptography          37.0.2
dacite                1.6.0
debugpy               1.6.0
decorator             5.1.1
defusedxml            0.7.1
docutils              0.18.1
entrypoints           0.4
executing             0.8.3
fastjsonschema        2.15.3
fire                

Let's start by importing useful modules

In [16]:
import yaml
from dataclasses import asdict
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from clap.utils import float_time_to_string, path_extend
from clap.executor import SSHCommandExecutor, AnsiblePlaybookExecutor

Let's create manager objects

In [17]:
configuration_db = get_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
# Private's path (usually ~/.clap/private/) will be used for other methods
private_path = node_manager.private_path

And get nodes in CLAP's system. Here we have 2 nodes, previously created with CLAP

In [18]:
nodes = node_manager.get_all_nodes()
for node in nodes:
    print(f"Node ID: {node.node_id} ({node.nickname}); status: {node.status}; "
          f"IP: {node.ip}; type: {node.configuration.instance.instance_config_id}")

Node ID: 7f0d4b44fdf7410a88358d0f2e2683bd (AlexArnold); status: started; IP: 34.238.151.159; type: type-t2.medium
Node ID: 12559354d6e140a2b195c137e3d3dbcb (MichelleRager); status: started; IP: 3.94.101.245; type: type-t2.medium


Let's update node information using the `is_alive` method

In [19]:
node_ids = [node.node_id for node in nodes]
for node_id, status in node_manager.is_alive(node_ids).items():
    print(f"Node {node_id} is {'alive' if status else 'not alive'}")

Node 7f0d4b44fdf7410a88358d0f2e2683bd is alive
Node 12559354d6e140a2b195c137e3d3dbcb is alive


## Executing Shell commands directly, using SSHCommandExecutor class

The SSHCommandExecutor class allows you to execute shel commands in nodes. The command is a string of shell commands. This class must be initializated with the command string, the list of **NodeDescriptors** and the path to private directory.

After using the run() method from SSHCommandExecutor, it wil be returned a dataclass called `CommandResult` with the following information:
* ok: if the command was executed in nodes (SSH performed and command executed)
* ret_code: if the command was executed (ok==True), this will contain the return code, otherwise None
* stdout_lines: if the command was executed (ok==True), this will contain a list of strings of the stdout output, otherwise None
* stderr_lines: if the command was executed (ok==True), this will contain a list of strings of the stderr output, otherwise None
* error: if the command was not executed (ok==False), this will contain the string with the exception, otherwise None

In [20]:
command_to_execute = """
git clone https://github.com/lmcad-unicamp/CLAP.git CLAP-ssh
echo Cloned CLAP into CLAP-ssh
"""
executor = SSHCommandExecutor(command_to_execute, nodes, private_path)
result = executor.run()

for node_id, res in result.items():
    print(f"Node id {node_id}, executed the command: {res.ok}, ret code: {res.ret_code}")
    # resut is a dataclass, we can convert to a dictionary
    res_dict = asdict(res)
    print('-----')
    # Dump dictionary in YAML format
    print(yaml.dump(res_dict, indent=4, sort_keys=True))

Node id 7f0d4b44fdf7410a88358d0f2e2683bd, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'Cloning into ''CLAP-ssh''...

    '
stdout_lines:
- 'Cloned CLAP into CLAP-ssh

    '

Node id 12559354d6e140a2b195c137e3d3dbcb, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'Cloning into ''CLAP-ssh''...

    '
stdout_lines:
- 'Cloned CLAP into CLAP-ssh

    '



# Executing an Ansible Playbook in nodes using the AnsiblePlaybookExecutor class

AnsiblePlaybookExecutor class allows CLAP to execute an Ansible playbook directly in CLAP's nodes. This class must be initializated with the path of the playbook, the path to private directory and the inventory.

The AnsiblePlaybookExecutor `run()` method executes the Playbook and returns a dataclass called `PlaybookResult`. 
This dataclass contains the following information:
* ok: if the command was executed in nodes (playbook was executed)
* ret_code: Ansible return code
* hosts: A dictionary where the keys are the node ids and the value is a boolean containing True if playbook was executed without errors in node and false otherwise.
* events: A dictionary where the keys are the node ids and the value is a list of dicionaries with all ansible events (see ansible-runner for more information on Ansible events)
* vars: A dictionary where the keys are the node ids and the value is a dictionary of the facts set to this node inside the playbook. So, every `set_fact` in Ansible Playbook is visible to CLAP through this variable. 

The inventory can be generated using AnsiblePlaybookExecutor.create_inventory static method, passing the nodes and the private path as arguments.

We will execute the playbook listed bellow at nodes: `07f4a369663f48d48254f2ad4c5abbfe` and `8133c6f7f9ca48258d1e0f01e11326f6`. The playbook can be used to install CLAP at nodes.

The playbook: 
* Update apt cache and install packages
* Clone a git repository
* Set CLAP's install.sh file to be executable
* Run install.sh script
* Set a fact called clap_dir with the directory where CLAP was installed at remote hosts. It will be visible to PlaybookResult.vars

Besides that, variables can be passed to playbook through 'extra' variable when initiaizing AnsiblePlaybookExecutor. This parameter receives a dictionary with key and values being strings.

In [21]:
!cat ~/playbook.yml

cat: /home/staff/edson/2022-mo833/ativ-9/mo833-atividade9/playbook.yml: No such file or directory


In [8]:
playbook_file = path_extend('~/playbook.yml')
inventory = AnsiblePlaybookExecutor.create_inventory(nodes, private_path)
executor = AnsiblePlaybookExecutor(playbook_file, private_path, inventory=inventory)
result = executor.run()

ERROR! the playbook: /home/staff/edson/2022-mo833/ativ-9/mo833-atividade9/playbook.yml could not be found


Let's check the playbook results

In [9]:
print(f"Did the playbook executed? {result.ok}")
print(f"Ansible playbook return code: {result.ret_code}")
print(f"Let's check how nodes executed: ")
for node_id, status in result.hosts.items():
    print(f"    Node {node_id}: {status}")
print(f"Let's check variables set using set_fact module: ")
for node_id, facts in result.vars.items():
    print(f"    Node {node_id}: {facts}")

Did the playbook executed? False
Ansible playbook return code: 1
Let's check how nodes executed: 
Let's check variables set using set_fact module: 


# Roles

Roles are easy ways to organize and execute Playbook at nodes that play a role. We will add nodes to a role called `commands-common`. 
Nodes that play this role can perform common operations like copy and fetch files, install and update packages, reboot, among others.

We will copy files from local to remote hosts, using commands-common role. The workflow will be:
* Add nodes to commands-common role
* Execute action calle copy on nodes of role commands-common

Let's see the commands common role at actions.d. The copy action requires the source file and the destiny where the files will be placed at remote hosts.

In [25]:
!cat ${CLAP_PATH}/roles/actions.d/commands-common.yml

---
actions:
  install-packages:
    playbook: roles/commands-common_install.yml
    description: Install packages in nodes
    vars:
    - name: packages
      description: Packages to install (comma separated)

  copy:
    playbook: roles/commands-common_copy.yml
    description: Copy files from localhost to remote hosts
    vars:
    - name: src
      description: Source files/directory to be copied
    - name: dest
      description: Destination directory where files will be placed

  fetch:
    playbook: roles/commands-common_fetch.yml
    description: Fetch files from remote hosts to localhosts
    vars:
    - name: src
      description: Source files/directory to be fetched
    - name: dest
      description: Destination directory where files will be placed

  reboot:
    playbook: roles/commands-common_reboot.yml
    description: Reboot a machine

  run-command:
    playbook: roles/commands-common_run-command.yml
    description: Run a shell command in remote hosts
    vars:
  

All roles can be accessed though the `roles` dictionary from the `role_manager` class, where the keys are the role names and values are the `Role` dataclasses. 
We will print all the roles found by the role_manager as a dict.

In [11]:
for role_name, role_info in role_manager.roles.items():
    # Convert role_info to a dict
    role_dict = asdict(role_info)
    print('------')
    print(f"Role: {role_name}")
    # Print role
    print(f"{yaml.dump(role_dict, indent=4)}")

------
Role: commands-common
actions:
    copy:
        description: Copy files from localhost to remote hosts
        playbook: roles/commands-common_copy.yml
        vars:
        -   description: Source files/directory to be copied
            name: src
            optional: false
        -   description: Destination directory where files will be placed
            name: dest
            optional: false
    fetch:
        description: Fetch files from remote hosts to localhosts
        playbook: roles/commands-common_fetch.yml
        vars:
        -   description: Source files/directory to be fetched
            name: src
            optional: false
        -   description: Destination directory where files will be placed
            name: dest
            optional: false
    install-packages:
        description: Install packages in nodes
        playbook: roles/commands-common_install.yml
        vars:
        -   description: Packages to install (comma separated)
            nam

Let's add the `commands-comon` role to all CLAP nodes on the `nodes_ids` list.

**Note**: If there is an action named `setup` defined in role, when using `add_role` from `RoleManager` class, this action will be automatically executed. The nodes will not be added to nodes if this action fails. If this action does not exists, the nodes will be added to role only.

In [27]:
added_nodes = role_manager.add_role('commands-common', node_ids)
print(f"Role commands-common was added to {len(added_nodes)} nodes: {node_ids}")

Role commands-common was added to 2 nodes: ['7f0d4b44fdf7410a88358d0f2e2683bd', '12559354d6e140a2b195c137e3d3dbcb']


Let's perform action update-packages at nodes and copy the file at `~/playbook.yml` to remote hosts, at `~`. 
The copy action requires `src` and `dest` vars to be informed. It will be informed through `extra_args` variable.

**Note**: An error will be raised if the variables are not informed 

In [31]:
copy_vars = {
    'src': path_extend('~/README.md'),
    'dest': '~'
}
playbook_result = role_manager.perform_action('commands-common', 'update-packages', node_ids)
playbook_result = role_manager.perform_action('commands-common', 'copy', node_ids, extra_args=copy_vars)

[WARNING]: Invalid characters were found in group names but not replaced, use
-vvvv to see details

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [7f0d4b44fdf7410a88358d0f2e2683bd]
ok: [12559354d6e140a2b195c137e3d3dbcb]

TASK [Perform package list update] *********************************************
changed: [7f0d4b44fdf7410a88358d0f2e2683bd]
changed: [12559354d6e140a2b195c137e3d3dbcb]

PLAY RECAP *********************************************************************
12559354d6e140a2b195c137e3d3dbcb : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
7f0d4b44fdf7410a88358d0f2e2683bd : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
[WARNING]: Invalid characters were found in group names but not replaced, use
-vvvv to see details

PLAY [all] ************************************************

We can also check all nodes that belong to a particular role using `get_all_role_nodes` method from `RoleManager` class.

If your role define hosts, you can use `get_all_role_nodes_hosts`.

In [32]:
nodes_belonging_to_role = role_manager.get_all_role_nodes('commands-common')
print(nodes_belonging_to_role)

['7f0d4b44fdf7410a88358d0f2e2683bd', '12559354d6e140a2b195c137e3d3dbcb']
